In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
terminos="/content/drive/My Drive/Seguimiento-Lucía/terminología/enlazado_llm/labour_law_literales.txt"
estatutos="/content/drive/My Drive/Seguimiento-Lucía/terminología/gold-standard/files"

In [ ]:
import os
import csv
import re

articulos_dir = estatutos
terminos_file = "/content/drive/My Drive/Seguimiento-Lucía/terminología/enlazado_llm/labour_law_literales.txt"
output_csv = '/content/drive/My Drive/Seguimiento-Lucía/terminología/enlazado_llm/terminos_contextos.csv'  # <-- Ruta de salida

with open(terminos_file, 'r', encoding='utf-8') as f:
    lista_terminos = [line.strip() for line in f if line.strip()]

# Inicializar resultados: diccionario
resultados = {termino: [] for termino in lista_terminos}

# Función para pluralizar la primera palabra
def pluralizar(termino):
    palabras = termino.split()
    primera = palabras[0]

    if primera.endswith(('a', 'e', 'i', 'o', 'u')):
        primera_plural = primera + 's'
    else:
        primera_plural = primera + 'es'

    palabras[0] = primera_plural
    return ' '.join(palabras)

# Función para encontrar contexto
def encontrar_contexto(texto, termino, num_palabras=20):
    contextos = []

    palabras = termino.split()
    patron = r'\b' + r'\s+'.join(map(re.escape, palabras)) + r'\b'

    for match in re.finditer(patron, texto, re.IGNORECASE):
        start = match.start()
        end = match.end()

        # 20 palabras antes
        antes = texto[:start]
        palabras_antes = antes.split()
        palabras_contexto = palabras_antes[-num_palabras:] if len(palabras_antes) >= num_palabras else palabras_antes

        # Hasta el primer punto
        despues = texto[end:]
        punto_final = despues.find('.')
        if punto_final != -1:
            despues_texto = despues[:punto_final + 1]
        else:
            despues_texto = despues

        contexto = ' '.join(palabras_contexto) + ' ' + texto[start:end] + ' ' + despues_texto
        contexto = contexto.replace('\n', ' ').strip()

        if len(contexto) > 500:
            contexto = contexto[:500] + '...'

        contextos.append(contexto)

    return contextos

# Procesar cada archivo
for i in range(1, 95):
    filename = f'articulo_{i}.txt'
    filepath = os.path.join(articulos_dir, filename)

    if not os.path.exists(filepath):
        print(f'Archivo no encontrado: {filename}')
        continue

    with open(filepath, 'r', encoding='utf-8') as f:
        texto = f.read()

    texto = texto.replace('\n', ' ').replace('\r', ' ')
    texto = re.sub(r'\s+', ' ', texto)

    for termino in lista_terminos:
        contextos = encontrar_contexto(texto, termino)

        # 🛑 Si no encuentra contextos, buscar en plural
        if not contextos:
            termino_plural = pluralizar(termino)
            contextos = encontrar_contexto(texto, termino_plural)

        resultados[termino].extend(contextos)

# Guardar en CSV
max_contextos = max(len(c) for c in resultados.values())

cabeceras = ['Termino'] + [f'Contexto_{i+1}' for i in range(max_contextos)]

with open(output_csv, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(cabeceras)

    for termino, contextos in resultados.items():
        fila = [termino] + contextos + [''] * (max_contextos - len(contextos))
        writer.writerow(fila)

print(f'✅ CSV generado correctamente: {output_csv} con {sum(len(c) for c in resultados.values())} contextos.')



✅ CSV generado correctamente: /content/drive/My Drive/Seguimiento-Lucía/terminología/enlazado_llm/terminos_contextos.csv con 2172 contextos.
